In [ ]:
# === F3Net on frames_cropped_faces_10src — print ONLY AUC, EER, AP ===
import os, re, glob, io, contextlib, warnings
warnings.filterwarnings("ignore")

from google.colab import drive
drive.mount('/content/drive', force_remount=False)

import numpy as np
import torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from sklearn.metrics import roc_auc_score, average_precision_score, roc_curve
import timm

# --- Paths ---
DRIVE_ROOT = "/content/drive/MyDrive" if os.path.exists("/content/drive/MyDrive") else "/content/drive/My Drive"
DATA_ROOT  = os.path.join(DRIVE_ROOT, "frames_cropped_faces_10src")   # {real,fake}
WEIGHT_PATH= os.path.join(DRIVE_ROOT, "DeepfakeBench_weights", "f3net_best.pth")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
IMG_SIZE = 299
IMAGENET_MEAN = torch.tensor([0.485, 0.456, 0.406]).view(1,3,1,1)
IMAGENET_STD  = torch.tensor([0.229, 0.224, 0.225]).view(1,3,1,1)

def pil_to_tensor(img: Image.Image, size=IMG_SIZE):
    if img.mode != "RGB":
        img = img.convert("RGB")
    if img.size != (size, size):
        img = img.resize((size, size), Image.BILINEAR)
    arr = np.asarray(img, dtype=np.float32) / 255.0   # HWC
    arr = np.transpose(arr, (2,0,1))                  # CHW
    t = torch.from_numpy(arr).unsqueeze(0)            # 1x3xHxW
    t = (t - IMAGENET_MEAN) / IMAGENET_STD
    return t.squeeze(0)  # 3xHxW

# ---------- F3Net’s FAD head ----------
def dct_matrix(size: int) -> torch.Tensor:
    i = torch.arange(size, dtype=torch.float32)
    j = torch.arange(size, dtype=torch.float32)
    jj, ii = torch.meshgrid(j, i, indexing='xy')
    mat = torch.cos((jj + 0.5) * torch.pi * ii / size)
    mat[0, :] = mat[0, :] * (1.0 / torch.sqrt(torch.tensor(size, dtype=torch.float32)))
    mat[1:, :] = mat[1:, :] * torch.sqrt(2.0 / torch.tensor(size, dtype=torch.float32))
    return mat.t()

def make_filter_mask(size, start, end):
    i = np.arange(size)
    j = np.arange(size)
    ii, jj = np.meshgrid(i, j, indexing='ij')
    s = ii + jj
    return ((s >= start) & (s <= end)).astype(np.float32)

class LearnableFilter(nn.Module):
    def __init__(self, size, band_start, band_end, learnable=True, normalize=False):
        super().__init__()
        self.base = nn.Parameter(torch.tensor(make_filter_mask(size, band_start, band_end)), requires_grad=False)
        self.learn = nn.Parameter(torch.randn(size, size) * 0.1, requires_grad=learnable)
        self.normalize = normalize
        if normalize:
            self.ft_num = nn.Parameter(torch.tensor(float(self.base.sum())), requires_grad=False)
    def forward(self, X):
        filt = self.base.to(X.device)
        if self.learn.requires_grad:
            filt = filt + (2.0 * torch.sigmoid(self.learn.to(X.device)) - 1.0)
        return X * (filt / self.ft_num if self.normalize else filt)

class FADHead(nn.Module):
    def __init__(self, size=IMG_SIZE):
        super().__init__()
        D = dct_matrix(size)
        self.D = nn.Parameter(D, requires_grad=False)
        self.DT = nn.Parameter(D.t(), requires_grad=False)
        low   = LearnableFilter(size, 0, int(size // 2.82))
        mid   = LearnableFilter(size, int(size // 2.82), size // 2)
        high  = LearnableFilter(size, size // 2, size * 2)
        allf  = LearnableFilter(size, 0, size * 2)
        self.filters = nn.ModuleList([low, mid, high, allf])
    def _dct2(self, x):   # (B,C,H,W)
        D, DT = self.D.to(x.device), self.DT.to(x.device)
        xh = torch.einsum('ih, b c h w -> b c i w', D, x)
        xw = torch.einsum('jw, b c i w -> b c i j', D, xh)
        return xw
    def _idct2(self, X):
        D, DT = self.D.to(X.device), self.DT.to(X.device)
        xw = torch.einsum('wj, b c i j -> b c i w', DT, X)
        xh = torch.einsum('hi, b c i w -> b c h w', DT, xw)
        return xh
    def forward(self, x):  # 3xHxW
        x = x.unsqueeze(0)
        X = self._dct2(x)
        outs = []
        for f in self.filters:
            Xp = f(X)
            yp = self._idct2(Xp)
            outs.append(yp)
        y = torch.cat(outs, dim=1)  # 1x12xHxW
        return y.squeeze(0)

class F3NetModel(nn.Module):
    def __init__(self, img_size=IMG_SIZE, num_classes=2):
        super().__init__()
        self.fad = FADHead(img_size)
        with contextlib.redirect_stdout(io.StringIO()), contextlib.redirect_stderr(io.StringIO()):
            self.backbone = timm.create_model("xception41", pretrained=True, num_classes=num_classes, in_chans=12)
        self.softmax = nn.Softmax(dim=1)
    def forward(self, x3):              # x3: (B,3,H,W)
        fad_feats = torch.stack([self.fad(x3[i]) for i in range(x3.size(0))], dim=0)  # (B,12,H,W)
        logits = self.backbone(fad_feats)  # (B,2)
        return logits

def try_load_weights(model, path):
    if not os.path.isfile(path):
        return False
    try:
        sd = torch.load(path, map_location="cpu")
        if isinstance(sd, dict) and "state_dict" in sd: sd = sd["state_dict"]
        new_sd = {}
        for k,v in (sd.items() if isinstance(sd, dict) else []):
            nk = k
            for pref in ("module.","model.","net.","backbone."):
                if nk.startswith(pref): nk = nk[len(pref):]
            new_sd[nk] = v
        model.load_state_dict(new_sd, strict=False)
        return True
    except Exception:
        return False

# ---------- Data (no torchvision) ----------
FRAME_KEY_RE = re.compile(r"^(.*?)(?:[_-]frames?[_-]?\d+|[_-]frame[_-]?\d+)$", re.IGNORECASE)
def get_video_key(basename):
    base = os.path.splitext(basename)[0]
    m = FRAME_KEY_RE.match(base)
    return m.group(1) if m else base.split("_")[0]

class FramesDataset(Dataset):
    def __init__(self, root):
        exts = {".jpg",".jpeg",".png",".bmp",".webp",".tif",".tiff",".JPG",".JPEG",".PNG"}
        self.samples=[]
        for cls,y in (("real",0),("fake",1)):
            d = os.path.join(root, cls)
            if not os.path.isdir(d): continue
            for p in glob.glob(os.path.join(d, "*")):
                if os.path.splitext(p)[1] in exts:
                    self.samples.append((p, y, get_video_key(os.path.basename(p))))
        self.samples.sort(key=lambda x:(x[1], x[2], x[0]))
    def __len__(self): return len(self.samples)
    def __getitem__(self, i):
        p,y,v = self.samples[i]
        with Image.open(p) as im:
            x = pil_to_tensor(im, IMG_SIZE)  # 3xHxW
        return x, y, v

def aggregate_by_video(vkeys, probs, labels, how="median"):
    vids={}
    for v,p,y in zip(vkeys, probs, labels):
        if v not in vids: vids[v]={"p":[], "y":y}
        vids[v]["p"].append(float(p))
    P=[]; Y=[]
    for v in vids:
        arr = np.array(vids[v]["p"], dtype=np.float32)
        P.append(float(np.median(arr)) if how=="median" else float(np.mean(arr)))
        Y.append(int(vids[v]["y"]))
    return np.array(P, dtype=np.float32), np.array(Y, dtype=np.int64)

def metrics_auc_eer_ap(y_true, y_score):
    auc = roc_auc_score(y_true, y_score)
    ap  = average_precision_score(y_true, y_score)
    fpr, tpr, thr = roc_curve(y_true, y_score)
    fnr = 1 - tpr
    idx = int(np.nanargmin(np.abs(fpr - fnr)))
    eer = float((fpr[idx] + fnr[idx]) / 2.0)
    return float(auc), float(eer), float(ap)

# --------------------- Run ---------------------
ds = FramesDataset(DATA_ROOT)
if len(ds)==0:
    raise RuntimeError(f"No images under {DATA_ROOT}/{{real,fake}}")

loader = DataLoader(ds, batch_size=16, shuffle=False, num_workers=2, pin_memory=True)

model = F3NetModel(img_size=IMG_SIZE).to(device).eval()
_ = try_load_weights(model, WEIGHT_PATH)
softmax = nn.Softmax(dim=1)

all_probs, all_labels, all_vkeys = [], [], []
with torch.no_grad():
    for xb, yb, vks in loader:
        xb = xb.to(device, dtype=torch.float32, non_blocking=True)
        logits = model(xb)
        probs = softmax(logits)[:,1].detach().cpu().numpy()
        all_probs.extend(probs.tolist())
        all_labels.extend(yb.numpy().tolist())
        all_vkeys.extend(list(vks))

all_probs  = np.asarray(all_probs, dtype=np.float32)
all_labels = np.asarray(all_labels, dtype=np.int64)
all_vkeys  = np.asarray(all_vkeys)

vp, vy = aggregate_by_video(all_vkeys, all_probs, all_labels, "median")
auc1, eer1, ap1 = metrics_auc_eer_ap(vy, vp)
auc2, eer2, ap2 = metrics_auc_eer_ap(vy, 1.0 - vp)
auc, eer, ap = (auc2, eer2, ap2) if auc2 > auc1 else (auc1, eer1, ap1)

print(f"AUC: {auc:.4f}")
print(f"EER: {eer:.4f}")
print(f"AP : {ap:.4f}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


model.safetensors:   0%|          | 0.00/108M [00:00<?, ?B/s]

AUC: 0.5280
EER: 0.4900
AP : 0.5843


In [ ]:
# =============== F3Net LARGE TABLE (frames_cropped_faces_10src) =================
# Columns:
# dataset, detector, video_name, true_label, n_frames, n_correct_frames, n_wrong_frames,
# frame_accuracy, avg_prob_fake, std_prob_fake, video_pred_by_avg, video_correct_by_avg,
# video_pred_by_majority, video_correct_by_majority

import os, re, glob, io, contextlib, warnings, sys, subprocess, math
warnings.filterwarnings("ignore")

# Mount Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

# --- Imports (no torchvision) ---
import numpy as np
import pandas as pd
import torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from sklearn.metrics import roc_auc_score, average_precision_score, roc_curve
import timm

# ---- Paths / names ----
DRIVE_ROOT   = "/content/drive/MyDrive" if os.path.exists("/content/drive/MyDrive") else "/content/drive/My Drive"
DATASET      = "frames_cropped_faces_10src"
DATA_ROOT    = os.path.join(DRIVE_ROOT, DATASET)            # {real,fake}
WEIGHT_PATH  = os.path.join(DRIVE_ROOT, "DeepfakeBench_weights", "f3net_best.pth")
DETECTOR     = "F3Net"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
IMG_SIZE = 299
IMAGENET_MEAN = torch.tensor([0.485, 0.456, 0.406]).view(1,3,1,1)
IMAGENET_STD  = torch.tensor([0.229, 0.224, 0.225]).view(1,3,1,1)

# ---- Tiny image pipeline (no torchvision) ----
def pil_to_tensor(img: Image.Image, size=IMG_SIZE):
    if img.mode != "RGB":
        img = img.convert("RGB")
    if img.size != (size, size):
        img = img.resize((size, size), Image.BILINEAR)
    arr = np.asarray(img, dtype=np.float32) / 255.0   # HWC -> [0,1]
    arr = np.transpose(arr, (2,0,1))                  # CHW
    t = torch.from_numpy(arr).unsqueeze(0)            # 1x3xHxW
    t = (t - IMAGENET_MEAN) / IMAGENET_STD
    return t.squeeze(0)  # 3xHxW

# ====================== F3Net’s FAD head (DCT + 4 bands) ======================
def dct_matrix(size: int) -> torch.Tensor:
    i = torch.arange(size, dtype=torch.float32)
    j = torch.arange(size, dtype=torch.float32)
    jj, ii = torch.meshgrid(j, i, indexing='xy')
    mat = torch.cos((jj + 0.5) * torch.pi * ii / size)
    mat[0, :]  *= (1.0 / torch.sqrt(torch.tensor(size, dtype=torch.float32)))
    mat[1:, :] *=  torch.sqrt(2.0 / torch.tensor(size, dtype=torch.float32))
    return mat.t()

def make_filter_mask(size, start, end):
    i = np.arange(size); j = np.arange(size)
    ii, jj = np.meshgrid(i, j, indexing='ij')
    s = ii + jj
    return ((s >= start) & (s <= end)).astype(np.float32)

class LearnableFilter(nn.Module):
    def __init__(self, size, band_start, band_end, learnable=True, normalize=False):
        super().__init__()
        self.base = nn.Parameter(torch.tensor(make_filter_mask(size, band_start, band_end)), requires_grad=False)
        self.learn = nn.Parameter(torch.randn(size, size) * 0.1, requires_grad=learnable)
        self.normalize = normalize
        if normalize:
            self.ft_num = nn.Parameter(torch.tensor(float(self.base.sum())), requires_grad=False)
    def forward(self, X):
        filt = self.base.to(X.device)
        if self.learn.requires_grad:
            filt = filt + (2.0 * torch.sigmoid(self.learn.to(X.device)) - 1.0)
        return X * (filt / self.ft_num if self.normalize else filt)

class FADHead(nn.Module):
    def __init__(self, size=IMG_SIZE):
        super().__init__()
        D = dct_matrix(size)
        self.D = nn.Parameter(D, requires_grad=False)
        self.DT = nn.Parameter(D.t(), requires_grad=False)
        low   = LearnableFilter(size, 0, int(size // 2.82))
        mid   = LearnableFilter(size, int(size // 2.82), size // 2)
        high  = LearnableFilter(size, size // 2, size * 2)
        allf  = LearnableFilter(size, 0, size * 2)
        self.filters = nn.ModuleList([low, mid, high, allf])
    def _dct2(self, x):   # (B,C,H,W)
        D, DT = self.D.to(x.device), self.DT.to(x.device)
        xh = torch.einsum('ih, b c h w -> b c i w', D, x)
        xw = torch.einsum('jw, b c i w -> b c i j', D, xh)
        return xw
    def _idct2(self, X):
        D, DT = self.D.to(X.device), self.DT.to(X.device)
        xw = torch.einsum('wj, b c i j -> b c i w', DT, X)
        xh = torch.einsum('hi, b c i w -> b c h w', DT, xw)
        return xh
    def forward(self, x):  # 3xHxW
        x = x.unsqueeze(0)
        X = self._dct2(x)
        outs = []
        for f in self.filters:
            Xp = f(X)
            yp = self._idct2(Xp)
            outs.append(yp)
        y = torch.cat(outs, dim=1)  # 1x12xHxW
        return y.squeeze(0)

# ====================== Backbone (timm xception, 12 in-channels) ======================
class F3NetModel(nn.Module):
    def __init__(self, img_size=IMG_SIZE, num_classes=2):
        super().__init__()
        self.fad = FADHead(img_size)
        with contextlib.redirect_stdout(io.StringIO()), contextlib.redirect_stderr(io.StringIO()):
            self.backbone = timm.create_model("xception41", pretrained=True, num_classes=num_classes, in_chans=12)
        self.softmax = nn.Softmax(dim=1)
    def forward(self, x3):              # x3: (B,3,H,W)
        fad_feats = torch.stack([self.fad(x3[i]) for i in range(x3.size(0))], dim=0)  # (B,12,H,W)
        logits = self.backbone(fad_feats)  # (B,2)
        return logits

def try_load_weights(model, path):
    if not os.path.isfile(path): return False
    try:
        sd = torch.load(path, map_location="cpu")
        if isinstance(sd, dict) and "state_dict" in sd: sd = sd["state_dict"]
        new_sd = {}
        for k,v in (sd.items() if isinstance(sd, dict) else []):
            nk=k
            for pref in ("module.","model.","net.","backbone."):
                if nk.startswith(pref): nk = nk[len(pref):]
            new_sd[nk]=v
        model.load_state_dict(new_sd, strict=False)
        return True
    except Exception:
        return False

# ====================== Data (group frames by video key) ======================
FRAME_KEY_RE = re.compile(r"^(.*?)(?:[_-]frames?[_-]?\d+|[_-]frame[_-]?\d+)$", re.IGNORECASE)
def get_video_key(basename):
    base = os.path.splitext(basename)[0]
    m = FRAME_KEY_RE.match(base)
    return m.group(1) if m else base.split("_")[0]

class FramesDataset(Dataset):
    def __init__(self, root):
        exts = {".jpg",".jpeg",".png",".bmp",".webp",".tif",".tiff",".JPG",".JPEG",".PNG"}
        self.samples=[]
        for cls,y in (("real",0),("fake",1)):
            d = os.path.join(root, cls)
            if not os.path.isdir(d): continue
            for p in glob.glob(os.path.join(d, "*")):
                if os.path.splitext(p)[1] in exts:
                    self.samples.append((p, y, get_video_key(os.path.basename(p))))
        self.samples.sort(key=lambda x:(x[1], x[2], x[0]))
    def __len__(self): return len(self.samples)
    def __getitem__(self, i):
        p,y,v = self.samples[i]
        with Image.open(p) as im:
            x = pil_to_tensor(im, IMG_SIZE)  # 3xHxW
        return x, y, v

# ====================== Metrics & thresholds ======================
def agg_by_video(vkeys, probs, labels, fn="median"):
    vids={}
    for v,p,y in zip(vkeys, probs, labels):
        if v not in vids: vids[v]={"p":[], "y":y}
        vids[v]["p"].append(float(p))
    names = sorted(vids.keys())
    P = np.array([np.median(vids[n]["p"]) if fn=="median" else np.mean(vids[n]["p"]) for n in names], dtype=np.float32)
    Y = np.array([vids[n]["y"] for n in names], dtype=np.int64)
    return names, P, Y

def youden_threshold(y_true, y_score):
    fpr, tpr, thr = roc_curve(y_true, y_score)
    j = tpr - fpr
    return float(thr[np.nanargmax(j)])

def lab2str(y): return "real" if int(y)==0 else "fake"

# ====================== Run inference ======================
ds = FramesDataset(DATA_ROOT)
if len(ds)==0:
    raise RuntimeError(f"No images under {DATA_ROOT}/{{real,fake}}")

loader = DataLoader(ds, batch_size=16, shuffle=False, num_workers=2, pin_memory=True)

model = F3NetModel(img_size=IMG_SIZE).to(device).eval()
_ = try_load_weights(model, WEIGHT_PATH)
softmax = nn.Softmax(dim=1)

frame_probs, frame_labels, frame_vkeys = [], [], []
with torch.no_grad():
    for xb, yb, vks in loader:
        xb = xb.to(device, dtype=torch.float32, non_blocking=True)
        logits = model(xb)
        p = softmax(logits)[:,1].detach().cpu().numpy()
        frame_probs.extend(p.tolist())
        frame_labels.extend(yb.numpy().tolist())
        frame_vkeys.extend(list(vks))

frame_probs  = np.asarray(frame_probs, dtype=np.float32)
frame_labels = np.asarray(frame_labels, dtype=np.int64)
frame_vkeys  = np.asarray(frame_vkeys)

# Auto orientation flip (choose p or 1-p maximizing VIDEO-level AUC under MEDIAN aggregation)
_, P_med, Y_vid = agg_by_video(frame_vkeys, frame_probs, frame_labels, "median")
auc_p  = roc_auc_score(Y_vid, P_med)
auc_1p = roc_auc_score(Y_vid, 1.0 - P_med)
if auc_1p > auc_p:
    frame_probs = 1.0 - frame_probs

# Thresholds:
thr_frame = youden_threshold(frame_labels, frame_probs)   # for per-frame + majority
names_avg, P_avg, Y_avg = agg_by_video(frame_vkeys, frame_probs, frame_labels, "mean")
thr_video_avg = youden_threshold(Y_avg, P_avg)            # for video_pred_by_avg

# Build per-video rows
rows = []
video_dict = {}
for v,p,y in zip(frame_vkeys, frame_probs, frame_labels):
    if v not in video_dict: video_dict[v] = {"probs": [], "label": int(y)}
    video_dict[v]["probs"].append(float(p))

for v in sorted(video_dict.keys()):
    probs = np.array(video_dict[v]["probs"], dtype=np.float32)
    y_int  = int(video_dict[v]["label"])
    y_str  = lab2str(y_int)
    n_frames = probs.size

    yhat_frames = (probs >= thr_frame).astype(int)
    n_correct_frames = int((yhat_frames == y_int).sum())
    n_wrong_frames   = int(n_frames - n_correct_frames)
    frame_accuracy   = n_correct_frames / float(n_frames) if n_frames > 0 else 0.0

    avg_prob_fake = float(np.mean(probs))
    std_prob_fake = float(np.std(probs))

    pred_avg_int = int(avg_prob_fake >= thr_video_avg)
    pred_avg_str = lab2str(pred_avg_int)
    video_correct_by_avg = int(pred_avg_int == y_int)

    pred_maj_int = int((yhat_frames.sum() >= math.ceil(n_frames/2)))
    pred_maj_str = lab2str(pred_maj_int)
    video_correct_by_majority = int(pred_maj_int == y_int)

    rows.append({
        "dataset": DATASET,
        "detector": DETECTOR,
        "video_name": v,
        "true_label": y_str,
        "n_frames": n_frames,
        "n_correct_frames": n_correct_frames,
        "n_wrong_frames": n_wrong_frames,
        "frame_accuracy": round(frame_accuracy, 4),
        "avg_prob_fake": round(avg_prob_fake, 6),
        "std_prob_fake": round(std_prob_fake, 6),
        "video_pred_by_avg": pred_avg_str,
        "video_correct_by_avg": video_correct_by_avg,
        "video_pred_by_majority": pred_maj_str,
        "video_correct_by_majority": video_correct_by_majority,
    })

df = pd.DataFrame(rows)

# Print full table (no truncation / no column breaks)
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 0)
print(df.to_string(index=False))
# ===============================================================================


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
                   dataset detector video_name true_label  n_frames  n_correct_frames  n_wrong_frames  frame_accuracy  avg_prob_fake  std_prob_fake video_pred_by_avg  video_correct_by_avg video_pred_by_majority  video_correct_by_majority
frames_cropped_faces_10src    F3Net       10_1       fake        20                 4              16            0.20       0.656579       0.022117              fake                     1                   real                          0
frames_cropped_faces_10src    F3Net      10_10       fake        20                12               8            0.60       0.680252       0.018214              fake                     1                   fake                          1
frames_cropped_faces_10src    F3Net      10_11       fake        20                 9              11            0.45       0.673700       0.016679          

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

import os, time
SAVE_DIR = "/content/drive/MyDrive/f3net results 10 src"
os.makedirs(SAVE_DIR, exist_ok=True)

CSV_PATH = os.path.join(SAVE_DIR, f"f3net_large_table_10src_{time.strftime('%Y%m%d-%H%M%S')}.csv")
df.to_csv(CSV_PATH, index=False)

print("Saved to:", CSV_PATH)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Saved to: /content/drive/MyDrive/f3net results 10 src/f3net_large_table_10src_20251021-163056.csv


In [ ]:
# === Small table from LARGE table `df` (use majority decision) ===
# correctly_predicted = "yes" if video_pred_by_majority == true_label else "no"

import pandas as pd
import numpy as np

# Normalize to lowercase strings just in case
tl = df["true_label"].astype(str).str.lower()
vp = df["video_pred_by_majority"].astype(str).str.lower()

small_df = df[["dataset","detector","video_name","true_label"]].copy()
small_df["correctly_predicted"] = np.where(vp == tl, "yes", "no")

# Print all rows with no column breaks
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 0)
print(small_df.to_string(index=False))


                   dataset detector video_name true_label correctly_predicted
frames_cropped_faces_10src    F3Net       10_1       fake                  no
frames_cropped_faces_10src    F3Net      10_10       fake                 yes
frames_cropped_faces_10src    F3Net      10_11       fake                  no
frames_cropped_faces_10src    F3Net      10_12       fake                 yes
frames_cropped_faces_10src    F3Net      10_13       fake                  no
frames_cropped_faces_10src    F3Net      10_14       fake                  no
frames_cropped_faces_10src    F3Net      10_15       fake                  no
frames_cropped_faces_10src    F3Net      10_16       fake                 yes
frames_cropped_faces_10src    F3Net      10_17       fake                 yes
frames_cropped_faces_10src    F3Net      10_18       fake                  no
frames_cropped_faces_10src    F3Net      10_19       fake                 yes
frames_cropped_faces_10src    F3Net       10_2       fake       

In [ ]:
import os, time
SAVE_DIR = "/content/drive/MyDrive/f3net results 10 src"
os.makedirs(SAVE_DIR, exist_ok=True)
small_df.to_csv(os.path.join(SAVE_DIR, f"f3net_small_table_majority_10src_{time.strftime('%Y%m%d-%H%M%S')}.csv"), index=False)
